# Newsvendor Problem with Demand Uncertainty

The newsvendor problem is a classical optimization problem in supply chain management and inventory control. It deals with the challenge of determining the optimal order quantity for a perishable product with uncertain demand. The problem is named after the scenario of a newsstand owner who needs to decide how many newspapers to order for the next day's business.

In the newsvendor problem, the retailer faces two types of costs:

- Holding cost: The cost incurred for storing unsold items at the end of the day.
- Shortage cost: The cost associated with not having enough inventory to meet customer demand, leading to lost sales or backorder penalties.

The optimal order quantity needs to balance these two costs while considering the uncertainty in demand. With perfect information about demand (which is never the case), the retailer could order the exact amount needed and avoid both costs. However, due to demand uncertainty, making the optimal decision is challenging. The newsvendor problem has applications in various industries, including retail, fashion, and perishable goods, where accurate demand forecasting is crucial for efficient inventory management and profitability.

This tutorial demonstrates how to formulate the newsvendor problem with demand uncertainty as a mathematical optimization problem, implement the problem using Pyomo, and solve it to obtain the optimal order quantity that minimizes the expected cost.

## Problem Formulation

Consider $D$ as a random variable representing the anticipated demand for the upcoming day. Each unit is procured at a cost of $c$. The expense associated with maintaining an item in inventory is represented by $h$, while $b$ denotes the penalty incurred when the inventory falls short of demand. The objective is to determine an order quantity, denoted by $x$, that minimizes the expected total cost.
The expected cost of ordering $x$ items is given by:

$$
\begin{align*}
\mathbb{E}\!\left[\text{cost}\right] & {}={} && c\,x + h\, \mathbb{E}\!\left[x - D\right]^+ + b\, \mathbb{E}\!\left[D - x\right]^+ \\
 & {}={} && \mathbb{E}\!\left[\max\{ (c + h)\, x - h\,D , (c - b)\, x + b\,D\}\right] 
\end{align*}
$$

## Reading the Historical Data

The historical data contains information about demand under various conditions, such as weather, day of the week, location, etc. Let's explore the features and visualize the demand distribution to better understand the data.

In [ ]:
# Install necessary libraries

!pip install numpy pandas matplotlib scikit-learn
!pip install gurobipy pyomo

In [ ]:
import numpy as np
import pandas as pd

# Read the historical data
hist = pd.read_csv("nv_hist_data_100.csv", index_col=0)

hist.head()

In [ ]:
# Print the number of features
print("Number of features:", hist.shape[1] - 1)

# See the demand statistics
hist["Demand"].describe()

In [ ]:
# Visualize the demand distribution
import matplotlib.pyplot as plt

plt.figure(figsize=(7, 5))
plt.hist(hist["Demand"], bins=20, edgecolor="black")
plt.title("Distribution of Demand")
plt.xlabel("Demand")
plt.ylabel("Frequency")
plt.show()

### Create Training and Testing Sets

In [ ]:
from sklearn.model_selection import train_test_split

# print the number of features
print("Num of features:", hist.shape[1] - 1)

In [ ]:
# Define independent and dependent variables
X = hist.iloc[:, :-1].values
Y = hist.iloc[:, -1].values

print("Features:")
print(X[:5], "...\n")
print("Demands:")
print(Y[:5], "...")

In [ ]:
# Split the data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=123)

## Model Construction

In [ ]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

solver_options = {}  # You can provide your WSL license information here
opt = SolverFactory("gurobi", solver_io="python", manage_env=True, solver_options=solver_options)

In [ ]:
# Define parameters
c = 10  # order cost
b = 15  # penalty cost
h = 1  # holding cost


def solve_newsvendor_sample_average(demands):
    """
    Solves the newsvendor problem using the sample average approximation given a list of demand samples.

    Parameters:
    - demands (list): A list of demand values for each sample.

    Returns:
    - obj (float): The objective value of the optimization problem.
    - sol (float): The optimal order quantity.

    """
    samples = range(len(demands))  # 0, 1, ..., N-1
    probability = 1 / len(demands)  # 1/N

    model = pyo.ConcreteModel()

    # Decision variables:
    model.x = pyo.Var(domain=pyo.NonNegativeReals)
    model.t = pyo.Var(samples, domain=pyo.NonNegativeReals)

    # Objective function
    model.cost = pyo.Objective(
        expr=sum(probability * model.t[i] for i in samples), sense=pyo.minimize
    )

    # Constraints
    model.cons = pyo.ConstraintList()
    for i in samples:
        model.cons.add(expr=model.t[i] >= (c + h) * model.x - h * demands[i])
        model.cons.add(expr=model.t[i] >= (c - b) * model.x + b * demands[i])

    # Solve the optimization problem
    opt.solve(model)

    # Get the objective value
    obj_val = pyo.value(model.cost)

    # Get the order quantity
    quantity = pyo.value(model.x)

    return obj_val, quantity

In [ ]:
def get_solution_cost(quantity, demand):
    """
    Calculates the cost of a solution for a newsvendor problem.

    Parameters:
    - quantity (int): The order quantity.
    - demand (int): The demand for the items.

    Returns:
    - total_cost (float): The cost of the solution.

    """
    total_cost = c * quantity + h * max(0, quantity - demand) + b * max(0, demand - quantity)
    return total_cost

## Solution Approaches

We will explore three different approaches to solve the newsvendor problem:

1. Expected Value Approximation (EVA): This approach uses the expected value (mean) of the demand distribution as the optimal order quantity. While simple, this approach may not perform well when the demand distribution is skewed or has significant variability.
2. Sample Average Approximation (SAA): This approach solves the newsvendor problem using a sample of demand realizations and takes the average of the optimal solutions.
3. Predict Then Optimize (PTO): In this approach, we first use a machine learning model (e.g., linear regression) to predict the demand based on the available features. Then, we solve the newsvendor problem using the predicted demand as the deterministic demand value. This approach may perform well if the prediction model can accurately capture the demand patterns.

### Approach 1: Expected Value Approximation

In [ ]:
demand_eva = np.mean(Y_train)
quantity_eva = demand_eva  # deterministic solution
obj_val_eva = get_solution_cost(quantity_eva, demand_eva)

print("Expected Value Approximation:")
print("Optimal cost:", obj_val_eva)
print("Optimal order quantity:", quantity_eva)

### Approach 2: Sample Average Approximation

This approach provides a more robust solution by considering the demand variability and minimizing the expected cost over the sample. However, it may require more computational resources, especially for large sample sizes.


In [ ]:
# We know the actual sample demands
obj_val_ssa, quantity_ssa = solve_newsvendor_sample_average(Y_train)

print("Sample Average Approximation:")
print("Optimal cost:", obj_val_ssa)
print("Optimal order quantity:", quantity_ssa)

### Approach 3: Predict Then Optimize

In this implementation, we use a linear regression model for simplicity, but other machine learning models, such as tree-based models, could be explored to potentially improve the predictive performance.

In [ ]:
from sklearn.linear_model import LinearRegression

# Run a linear regression model to predict the demand
ml = LinearRegression()
ml.fit(X_train, Y_train)

# Predict the demand
Y_pred = ml.predict(X_test)

Y_pred

In [ ]:
solutions_pred = [demand_pred for demand_pred in Y_pred]  # deterministic solutions

### Calculate Errors

In [ ]:
def calculate_error(true_demands, quantities):
    """
    Calculates the relative error between the true costs and the estimated costs for the given solutions.

    Parameters:
    - true_demands (list): A list of actual demand values.
    - quantities (list): A list of order quantities corresponding to the demand values.

    Returns:
    - relative_error (float): The relative error between the true costs and the estimated costs.

    """
    true_solutions = true_demands
    true_costs = np.array(
        [
            get_solution_cost(solution, demand)
            for solution, demand in zip(true_solutions, true_demands)
        ]
    )
    estimated_costs = np.array(
        [get_solution_cost(quantity, demand) for quantity, demand in zip(quantities, true_demands)]
    )

    diffs = np.sum(np.abs(true_costs - estimated_costs))
    relative_error = diffs / np.sum(true_costs)

    return relative_error

The relative error metric calculates the absolute difference between the true costs and the estimated costs, and then normalizes it by the sum of the true costs. This metric provides a measure of how well the different approaches perform in terms of minimizing the total cost.

In [ ]:
# Expected value approximation
quantities = [quantity_eva] * len(Y_test)  # same order quantity for all upcoming days
error_eva = calculate_error(Y_test, quantities)

# Sample average approximation
quantities = [quantity_ssa] * len(Y_test)  # same order quantity for all upcoming days
error_ssa = calculate_error(Y_test, quantities)

# Prediction
quantities = solutions_pred  # predicted quantity for each upcoming day
error_pred = calculate_error(Y_test, quantities)

print("Expected value approximation error: {:.1%}".format(error_eva))
print("Sample average approximation error: {:.1%}".format(error_ssa))
print("Linear regression prediction error: {:.1%}".format(error_pred))

### Exercise

Implement the *Predict-Then-Optimize* approach using a decision tree regressor.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

ml = DecisionTreeRegressor()
ml.fit(X_train, Y_train)

# Predict the demand
Y_pred = ml.predict(X_test)

solutions_pred = [demand_pred for demand_pred in Y_pred]  # deterministic solutions

print(solutions_pred)

In [ ]:
quantities = solutions_pred  # predicted quantity for each upcoming day
error_pred = calculate_error(Y_test, quantities)

print("Decision tree prediction error: {:.1%}".format(error_pred))